# Coral Classification Challenge

File name: CoralAI.ipynb

Author: kogni7

Date: October 2021

## Contents

* 1 Preparation
* 2 Data
* 3 Training
* 4 Prediction and Submission

This notebook uses only the data sets provided by ZINDI. These data sets contain images of corals. These are the only used features in this notebook. The task is to classify the images.

The file system for this project is:

* CoralAI (root)
    * CoralAI.ipynb (this notebook)
    * Data
        * coral_image.zip
        * Train.csv
        * Test.csv
        * SampleSubmission.csv
    * Submission
        * 1 - x: Submission directories named by the version number
            * submission.csv

This jupyter notebook runs in Google Colab without special configuration. GPU is enabled.

This notebook uses a Transfer learning based approach.

Reference: https://github.com/kogni7git/solutions/blob/main/PAYGoAI.ipynb


## 1 Preparation

In [1]:
import time
start_time = time.time()

### Installation, Libraries and Seed

In [2]:
# Installation
!pip install efficientnet_pytorch

# Seed, Libraries
SEED = 42

# Math
import numpy as np
print("Numpy Version: " + str(np.__version__))

import random
import os
os.environ['PYTHONHASHSEED'] = str(SEED)

np.random.seed(SEED)

random.seed(SEED)

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
print("PyTorch Version: " + str(torch.__version__))
torch.manual_seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = False

import torchvision
from torchvision import datasets, transforms
print("PyTorch torchvision Version: " + str(torchvision.__version__))

# EfficientNet
import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
print("efficientnet_pytorch Version: " + str(efficientnet_pytorch.__version__))

# CSV
import pandas as pd
print("Pandas Version: " + str(pd.__version__))

# Machine Learning
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
print("SciKit-Learn Version: " + str(sklearn.__version__))

# Images
import PIL
from PIL import Image
print("PIL Version: " + str(PIL.__version__))

from tqdm import tqdm
import gc

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=83ff2ee9eff304fd7ad6f2c1031a90205e07da6a5a5540854b7eaade5dcb63f0
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
Numpy Version: 1.19.5
PyTorch Version: 1.9.0+cu111
PyTorch torchvision Version: 0.10.0+cu111
efficientnet_pytorch Version: 0.7.1
Pandas Version: 1.1.5
SciKit-Learn Version: 0.22.2.post1
PIL Version: 7.1.2


In [3]:
!nvidia-smi

Sun Oct 31 15:53:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Parameters

In [4]:
CLASSES = 5

EPOCHS = 5

BATCH_SIZE = 32

TEST_BATCH_SIZE = 128

"""
b0: 224
"""
NET = 'efficientnet-b0'

IMAGE_SIZE = 224

CV = 2

VERBOSE = True

LEARNING_RATE = 0.5e-4
SCHEDULER = False

# The Version
VERSION = "TEST"

# for use in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
WD = os.getcwd() + "/drive/My Drive/CoralAI"
print(WD)

/content/drive/My Drive/CoralAI


In [6]:
!unzip '/content/drive/My Drive/CoralAI/Data/coral_image.zip'

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: coral_image/ImageID_57I10QK8.jpg  
  inflating: __MACOSX/coral_image/._ImageID_57I10QK8.jpg  
  inflating: coral_image/ImageID_CKA4ZOKX.jpg  
  inflating: __MACOSX/coral_image/._ImageID_CKA4ZOKX.jpg  
  inflating: coral_image/ImageID_Z2BSY7TR.jpg  
  inflating: __MACOSX/coral_image/._ImageID_Z2BSY7TR.jpg  
  inflating: coral_image/ImageID_3KR33EMZ.jpg  
  inflating: __MACOSX/coral_image/._ImageID_3KR33EMZ.jpg  
  inflating: coral_image/ImageID_WTH9HU7X.jpg  
  inflating: __MACOSX/coral_image/._ImageID_WTH9HU7X.jpg  
  inflating: coral_image/ImageID_9E3Y83E6.jpg  
  inflating: __MACOSX/coral_image/._ImageID_9E3Y83E6.jpg  
  inflating: coral_image/ImageID_9LI4WQBM.jpg  
  inflating: __MACOSX/coral_image/._ImageID_9LI4WQBM.jpg  
  inflating: coral_image/ImageID_3ZW7JUXC.jpg  
  inflating: __MACOSX/coral_image/._ImageID_3ZW7JUXC.jpg  
  inflating: coral_image/ImageID_VRWSSZND.jpg  
  inflating: __MACOSX/coral_i

In [7]:
len(os.listdir("coral_image"))

33643

## 2 Data

In [8]:
transformations = transforms.Compose([transforms.ToTensor(), transforms.Resize((IMAGE_SIZE, IMAGE_SIZE))])

In [9]:
train_csv = pd.read_csv(WD + "/Data/Train.csv")
train_csv.head()

,Image_ID,label
0,ImageID_001G9EFI,Other
1,ImageID_0026LXMN,Hard Coral
2,ImageID_004J6Y6A,Hard Coral
3,ImageID_0056CP11,Hard Coral
4,ImageID_00AV0G9B,Other


In [10]:
test_csv = pd.read_csv(WD + "/Data/Test.csv")
test_csv.head()

,Image_ID
0,ImageID_001KGM1C
1,ImageID_0048JTR3
2,ImageID_00491SIG
3,ImageID_004B1V2U
4,ImageID_00C0Q21B


In [11]:
sample_submission_csv = pd.read_csv(WD + "/Data/SampleSubmission.csv")
sample_submission_csv.head()

,Image_ID,label
0,ImageID_001KGM1C,NaN
1,ImageID_0048JTR3,NaN
2,ImageID_00491SIG,NaN
3,ImageID_004B1V2U,NaN
4,ImageID_00C0Q21B,NaN


In [12]:
encoder = LabelEncoder()

train_labels = encoder.fit_transform(np.array(train_csv.label))
train_labels

array([2, 1, 1, ..., 0, 1, 0])

In [13]:
class MakeDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __getitem__(self, idx):
        return {'image': transformations(Image.open("coral_image/" + self.images.Image_ID[idx] + ".jpg")),
                'label': torch.tensor(self.labels[idx])}

    def __len__(self):
        return len(self.images)

"""
train_ = MakeDataset(train_csv, train_labels, "Training")
test_ = MakeDataset(train_csv.iloc[:30], None, "Test")
print(train_[1])
print(test_[1])

d = torch.utils.data.DataLoader(train_, batch_size=2)
for i in d:
    print(i['label'])
"""

'\ntrain_ = MakeDataset(train_csv, train_labels, "Training")\ntest_ = MakeDataset(train_csv.iloc[:30], None, "Test")\nprint(train_[1])\nprint(test_[1])\n\nd = torch.utils.data.DataLoader(train_, batch_size=2)\nfor i in d:\n    print(i[\'label\'])\n'


This is an efficient net with a linear layer and sigmoidal activation.

In [14]:
class EfficientNeuralNet(nn.Module):
    def __init__(self, net):
        super(EfficientNeuralNet, self).__init__()
        self.net = EfficientNet.from_pretrained(net)
        self.ffn1 = nn.Linear(self.net._fc.out_features, 128)
        self.ffn2 = nn.Linear(128, CLASSES)
        self.relu = nn.ReLU()
        self.soft = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.net(x)
        x = self.ffn1(x)
        x = self.relu(x)
        return self.soft(self.ffn2(x))

## 3 Training

In [15]:
def training(data_loader, model, optimizer):

    model.train()

    losses = []
    accuracies = []

    for data in tqdm(data_loader):
        image = data['image'].cuda() / 255
        target = data['label'].cuda()

        optimizer.zero_grad()

        output = model(image.float())

        loss = nn.CrossEntropyLoss()(output.squeeze().float(), target.long())

        loss.backward()
        optimizer.step()

        losses.append(loss.data.cpu())
        accuracies.append(accuracy_score(target.data.cpu().numpy(), torch.argmax(output, dim=1).data.cpu().numpy())) 

        del output, loss, image, target
        gc.collect()

 
    return np.mean(losses), np.mean(accuracies)


def evaluation(data_loader, model):
    
    model.eval()
    
    losses = []
    accuracies = []

    with torch.no_grad():
        for data in tqdm(data_loader):
            image = data['image'].cuda() / 255
            target = data['label'].cuda()

            output = model(image.float())
            loss = nn.CrossEntropyLoss()(output.squeeze().float(), target.long())

            losses.append(loss.data.cpu())
            accuracies.append(accuracy_score(target.data.cpu().numpy(), torch.argmax(output, dim=1).data.cpu().numpy())) 

            del output, loss, image, target
            gc.collect()

    return np.mean(losses), np.mean(accuracies)


def prediction(data_loader, model):
    model.eval()
    
    outputs = []

    with torch.no_grad():
        for data in tqdm(data_loader):
            image = data['image'].cuda() / 255

            output = model(image.float())
            outputs.append(output.squeeze().data.cpu())

            del output, image
            gc.collect()

        return torch.cat(outputs)

In [16]:
def seed_worker(worker_id):
    """
    https://pytorch.org/docs/stable/notes/randomness.html
    """
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


cv = 1

STATISTICS = {}
STATISTICS['TRAIN'] = np.zeros((EPOCHS, CV))
STATISTICS['TRAIN_ACC'] = np.zeros((EPOCHS, CV))
STATISTICS['VALIDATION'] = np.zeros((EPOCHS, CV))
STATISTICS['VALIDATION_ACC'] = np.zeros((EPOCHS, CV))

for train, val in StratifiedKFold(n_splits=CV, shuffle=True, random_state=SEED).split(train_csv, train_csv.label):
    print("Run {} of {}.".format(cv, CV))

    # Data
    g = torch.Generator()
    g.manual_seed(SEED)
    TRAIN_SET = MakeDataset(train_csv.iloc[train].reset_index(), train_labels[train])
    train_data_loader = torch.utils.data.DataLoader(TRAIN_SET, batch_size=BATCH_SIZE, worker_init_fn=seed_worker, generator=g, num_workers=2)

    g = torch.Generator()
    g.manual_seed(SEED)
    VALIDATION_SET = MakeDataset(train_csv.iloc[val].reset_index(), train_labels[val])
    val_data_loader = torch.utils.data.DataLoader(VALIDATION_SET, batch_size=TEST_BATCH_SIZE, worker_init_fn=seed_worker, generator=g, num_workers=2)

    # Model
    model = EfficientNeuralNet(NET)
    model = model.cuda()

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    if SCHEDULER:
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

    BEST_Acc = 0.0

    for epoch in range(EPOCHS):
        train_loss, train_acc = training(train_data_loader, model, optimizer)
        val_loss, val_acc = evaluation(val_data_loader, model)

        if SCHEDULER:
            scheduler.step()

        if val_acc > BEST_Acc:
            torch.save(model.state_dict(), '/content/model_' + str(cv) + '.pt')
            BEST_Acc = val_acc

        if VERBOSE:
            print("Epoch: {}; TRAINING: {:.3f}; Accuracy: {:.3f}; VALIDATION: {:.3f}; Accuracy: {:.3f}".format(epoch, train_loss, train_acc, val_loss, val_acc))

        STATISTICS['TRAIN'][epoch, cv-1] = train_loss
        STATISTICS['TRAIN_ACC'][epoch, cv-1] = train_acc
        STATISTICS['VALIDATION'][epoch, cv-1] = val_loss
        STATISTICS['VALIDATION_ACC'][epoch, cv-1] = val_acc

    del BEST_Acc, train_loss, val_loss, train_acc, val_acc, model, train_data_loader, val_data_loader
    gc.collect()
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

    print("\n")
    cv += 1

print("Result:")
for epoch in range(EPOCHS):
    print("Epoch: {}; TRAINING: {:.3f}; Accuracy: {:.3f}; VALIDATION: {:.3f}; Accuracy: {:.3f}".format(epoch, 
                                                                                                       np.mean(STATISTICS['TRAIN'][epoch, :]),
                                                                                                       np.mean(STATISTICS['TRAIN_ACC'][epoch, :]),
                                                                                                       np.mean(STATISTICS['VALIDATION'][epoch, :]),
                                                                                                       np.mean(STATISTICS['VALIDATION_ACC'][epoch, :])))

Run 1 of 2.


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b0


100%|██████████| 85/85 [01:06<00:00,  1.29it/s]


...

Epoch: 2; TRAINING: 1.247; Accuracy: 0.656; VALIDATION: 1.322; Accuracy: 0.576


100%|██████████| 85/85 [01:06<00:00,  1.29it/s]


Epoch: 3; TRAINING: 1.218; Accuracy: 0.687; VALIDATION: 1.311; Accuracy: 0.590


100%|██████████| 85/85 [01:06<00:00,  1.29it/s]


Epoch: 4; TRAINING: 1.202; Accuracy: 0.701; VALIDATION: 1.310; Accuracy: 0.590


Result:
Epoch: 0; TRAINING: 1.364; Accuracy: 0.557; VALIDATION: 1.508; Accuracy: 0.408
Epoch: 1; TRAINING: 1.289; Accuracy: 0.613; VALIDATION: 1.567; Accuracy: 0.313
Epoch: 2; TRAINING: 1.246; Accuracy: 0.658; VALIDATION: 1.314; Accuracy: 0.585
Epoch: 3; TRAINING: 1.215; Accuracy: 0.689; VALIDATION: 1.309; Accuracy: 0.592
Epoch: 4; TRAINING: 1.196; Accuracy: 0.708; VALIDATION: 1.308; Accuracy: 0.592


## 4 Prediction and Submission

In [17]:
sample_submission_csv.label = 0

predictions = torch.zeros(len(sample_submission_csv), CLASSES)

for cv in range(CV):
    # Data
    g = torch.Generator()
    g.manual_seed(SEED)
    TEST_SET = MakeDataset(sample_submission_csv, np.array(sample_submission_csv.label))
    test_data_loader = torch.utils.data.DataLoader(TEST_SET, batch_size = TEST_BATCH_SIZE, worker_init_fn=seed_worker, generator=g, num_workers=2)

    # Model
    model = EfficientNeuralNet(NET)
    model.load_state_dict(torch.load('/content/model_' + str(cv + 1) + '.pt'))
    model = model.cuda()

    # Prediction
    outputs = prediction(test_data_loader, model)
    predictions += outputs

    del model, TEST_SET, test_data_loader, outputs

    gc.collect()
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

    predictions /= CV

predictions = torch.argmax(predictions, dim=1)
sample_submission_csv.label = list(encoder.inverse_transform(predictions))

os.mkdir(WD + '/Submission/' + str(VERSION)) 
sample_submission_csv.to_csv(WD + '/Submission/' + str(VERSION) + '/submission.csv', index=False)

Loaded pretrained weights for efficientnet-b0


100%|██████████| 94/94 [01:12<00:00,  1.29it/s]


Loaded pretrained weights for efficientnet-b0


100%|██████████| 94/94 [01:12<00:00,  1.29it/s]


In [18]:
sample_submission_csv

,Image_ID,label
0,ImageID_001KGM1C,Algae
1,ImageID_0048JTR3,Algae
2,ImageID_00491SIG,Algae
3,ImageID_004B1V2U,Algae
4,ImageID_00C0Q21B,Algae
...,...,...
11958,ImageID_ZZP1Y9Z1,Hard Coral
11959,ImageID_ZZPHNP5L,Algae
11960,ImageID_ZZUIMD0P,Algae
11961,ImageID_ZZUXL8E9,Algae


In [19]:
drive.flush_and_unmount()

In [20]:
end_time = time.time()
print("Runtime of the Notebook: {} min".format(np.round((end_time - start_time) / 60, 2)))

Runtime of the Notebook: 56.9 min
